# Signal CSV

notebook for exploring csv files collected in the field

there are three ways to select an npz to analyze

1. specifying the local path
2. using the upload widget 
3. running from the [Runner Notebook](runner.ipynb)

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Layout, interact_manual

import cionic
from cionic import tools, triggers

## Parameters


These parameters will be overridden when this notebook is used with the runner

```
zippath     path on your local filesystem to find the recording or path
            or path on the local filesystem to store the recording if download set
            
download    xid of collection to download

tokenpath   path to auth token if download is set

```

In [ ]:
# param

files_url = None
files_dir = None
tokenpath = None

In [ ]:
if files_url:
    cionic.auth(tokenpath=tokenpath)
    files = cionic.download_files(files_url, files_dir, include=[".csv"])
else:
    files = cionic.list_files(files_dir, include=[".csv"])

files

## Interface

The following elements can be edited via the interface

1. which stream to examine
2. which filter to run on the signal
3. the cutoff frequency of that filter
4. window size for rms
5. fft or cdf


In [ ]:
frames = {}

for file in files:
    (name, ext) = file.split(".")
    if ext == "csv":
        if files_dir:
            path = f"{files_dir}{file}"
        else:
            path = file
        if name == "imu":
            frames[name] = tools.csv_imu_convert(pd.read_csv(path))
        else:
            frames[name] = pd.read_csv(path)

streams = list(frames.keys())
streams

In [ ]:
# use IMU times to sync shank signals to thigh signals
# todo: does not work for dual sleeve
imuf = frames["imu"]
mins = {}

for limb in ["r_shank", "r_thigh", "l_shank", "l_thigh"]:
    tf = imuf[imuf["limb"] == limb]
    limb_min = np.min(tf['elapsed'])
    if not np.isnan(limb_min):
        mins[limb] = limb_min

if "r_shank" in mins and "r_thigh" in mins:
    offset = mins["r_thigh"] - mins["r_shank"]
    print(f"Updating right shank signals with {offset}")

    # update right shank streams to match dc frames
    frames['imu'].loc[frames['imu'].limb == "r_shank", "elapsed"] = (
        frames['imu'].loc[frames['imu'].limb == "r_shank", "elapsed"] + offset
    )
    # todo: only update emg for NS100
    frames['emg'].loc[frames['emg'].limb == "r_shank_emg", "elapsed"] = (
        frames['emg'].loc[frames['emg'].limb == "r_shank_emg", "elapsed"] + offset
    )

if "l_shank" in mins and "l_thigh" in mins:
    offset = mins["l_thigh"] - mins["l_shank"]
    print(f"Updating left shank signals with {offset}")

    # update left shank streams to match dc frames
    frames['imu'].loc[frames['imu'].limb == "l_shank", "elapsed"] = (
        frames['imu'].loc[frames['imu'].limb == "l_shank", "elapsed"] + offset
    )
    # todo: only update emg for NS100
    frames['emg'].loc[frames['emg'].limb == "l_shank_emg", "elapsed"] = (
        frames['emg'].loc[frames['emg'].limb == "l_shank_emg", "elapsed"] + offset
    )

In [ ]:
times = [np.min(frames['emg']['elapsed']), np.max(frames['emg']['elapsed'])]
times

In [ ]:
components = []
excluded = ['limb', 'elapsed']

for stream, frame in frames.items():
    if 'limb' in frame:
        for limb in set(frame['limb']):
            for comp in [x for x in frame.keys() if x not in excluded]:
                components.append(f"{stream} {limb} {comp}")
    elif stream == 'action':
        action_frame = triggers.get_action_csv(frame)
        for m in set(zip(action_frame["muscle"], action_frame["name"])):
            components.append(f"stim {m[0]} {m[1]}")
    else:
        for comp in [x for x in frame.keys() if x not in excluded]:
            components.append(f"{stream} {comp}")

In [ ]:
w = '400px'
sliderLayout = Layout(width=w)
dropdownLayout = Layout(width=w, height='200px')

sp = widgets.SelectMultiple(
    options=components, description="streams", layout=dropdownLayout
)
rs = widgets.FloatRangeSlider(
    min=times[0],
    max=times[1],
    value=times,
    continuous_update=False,
    description="times",
    layout=sliderLayout,
)
a1 = widgets.BoundedIntText(
    value=1, min=0, max=100000, step=1000, description="axis 1 scale"
)
a2 = widgets.BoundedIntText(
    value=10000, min=0, max=100000, step=1000, description="axis 2 scale"
)
title = widgets.Text(
    value='', placeholder='Enter title here', description='plot title:', disabled=False
)
xlabel = widgets.Text(
    value='',
    placeholder='Enter xlabel here',
    description='x-axis label:',
    disabled=False,
)
ylabel = widgets.Text(
    value='',
    placeholder='Enter ylabel here',
    description='y-axis label:',
    disabled=False,
)
color = widgets.Text(
    value='',
    placeholder='This will override plot colors.',
    description='plot color:',
    disabled=False,
)
ylim_min = widgets.FloatText(value=0, description='ylim min', disabled=False)
ylim_max = widgets.FloatText(value=0, description='ylim max', disabled=False)
plot_same = widgets.Checkbox(value=False, description='Same Plot?')
ncols = widgets.IntText(value=1, description='# columns', disabled=False)
legend_loc = widgets.Dropdown(
    options=[
        'best',
        'upper right',
        'upper left',
        'lower right',
        'lower left',
        'upper center',
        'lower center',
    ],
    description="legend location",
)
plot_sharex = widgets.Checkbox(value=True, description='Share x?')


@interact_manual
def show_emg(
    stream=sp,
    times=rs,
    style=['-', 'o'],
    a1_scale=a1,
    a2_scale=a2,
    plot_title=title,
    plot_xlabel=xlabel,
    plot_ylabel=ylabel,
    plot_color=color,
    plot_min=ylim_min,
    plot_max=ylim_max,
    legend_location=legend_loc,
    plot_on_same=plot_same,
    sharex=plot_sharex,
    plot_ncols=ncols,
):

    # close all the plots
    plt.close('all')

    # translate the stim streams for shades
    (muscles, stream) = triggers.check_for_action_csv(stream)
    stims = triggers.compute_stim_muscles(action_frame, muscles, times=times)

    # Override xaxis and yaxis labels
    # NOTE: Not to be used with "Same Plot?" checked - this will not work
    # NOTE: length of these lists MUST be the same as the number of streams selected
    # plot_xlabel = ['This is the first xlabel', 'This is the second', 'This is the third']
    # plot_ylabel = ['This is the first ylabel', 'This is the second', 'This is the third']
    stream_list = list(stream)
    stream_list.sort()
    signals, legends = tools.csv_signals(
        frames, stream_list, times, [a1_scale, a2_scale]
    )

    # plot filtered signal, rms, and fft
    # signals, legends = tools.compute_signals(data_dict, regs_dict, stream, times, fil, rms, fft, [a1_scale,a2_scale])
    # tools.simple_plot(signals, leg_contents = legends, y_column="elapsed", title='Signal', style=style)
    tools.configurable_plot(
        signals,
        leg_contents=legends,
        y_column='elapsed',
        title=plot_title,
        xlabel=plot_xlabel,
        ylabel=plot_ylabel,
        ylim=[ylim_min.value, ylim_max.value],
        color=plot_color,
        style=style,
        ncols=plot_ncols,
        same_plot=plot_on_same,
        legend_loc=legend_location,
        sharex=sharex,
        shades=stims,
    )